# Origen de los textos:

Los textos que se encuentran cargados en el el excel que se utiliza provienen de hojas de especificación de luminarias como la que se puede obsevar en el siguiente link: [Hoja de especificación](https://www.cooperlighting.com/api/assets/v1/file/CLS/content/metalux-skyridge-selectable-specsheet/Metalux-SkyRidge-LED-Selectable-spec-sheet.pdf)

In [1]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.feature_extraction.text import CountVectorizer

En este caso yo necesitaba realizar varios conteos de ngramas por lo que cree una función que retornara "bag of words" de n-gramas en forma de un data frame:

In [2]:
#Function to extract the text from PDF and transform it into a bag of words data frame

def PDFtextextract_tobow (pdfs_texts, n_grams):
    #pdfs_texts---->Input: list of PDF texts
    #pdfs_bow_df---->Returned value: Data frame of the pdfs bag of words
    
    vectorizer = CountVectorizer(stop_words='english', ngram_range= n_grams)
    vectorizer.fit(pdfs_texts)
    pdfs_bow = vectorizer.transform(pdfs_texts)
    pdfs_bow_df = pd.DataFrame(pdfs_bow.toarray(), columns=vectorizer.get_feature_names_out())
    
    return pdfs_bow_df

Tambien necesitaba poder ver en cuantos documentos aparecia cada bigrama por lo que hice una función que me permitiera tener el topn de bigramas con mas documentos

Esta función tambien me permite que palabras quedarme del topn.

In [3]:
def BOWreduce(bow_df, Topn_words):
    #bow_df---->Input: Bag of Words dataframe
    #Topn_words---->Input: int of number of n words with most document appereances
    #bow_reduced---->Returned value: Data frame of the bag of words containing just the selected words

    document_appearances = bow_df[bow_df > 0].count() #Counts num of documents a word appears in
    top_document_appearances = document_appearances.nlargest(Topn_words, keep = 'all') #Gets the top n most appeared words
    top_words = top_document_appearances.index #Saves the top n words in a lst

    #Iterates through the words
    for word in top_words:
        keep_word = input('Keep "' + word + '"?  ') #Ask for user input if he wants to keep or remove the word

        #"y" means we want to keep the word any other series of characters means we want to delete it
        if keep_word != 'y': 
            top_document_appearances = top_document_appearances.drop(labels = [word]) #Drops word

    print(top_document_appearances)

    top_words = top_document_appearances.index #Gets the words that were kept
    bow_reduced = bow_df[top_words[0]] #Creates dataframe with the kept words
    top_words
    top_document_appearances = top_document_appearances.drop(labels = [top_words[0]]) #Drops item used to create dataframe
    top_words = top_document_appearances.index #Gets words without the dropped word

    for word in top_words:
        bow_reduced = pd.concat([bow_reduced, bow_df[word]], axis = 1) #Adds the pending words to the dataframe


    return bow_reduced

Se lee el excel con los textos que se encuentran en la columna "pdf_texts":

In [4]:
#Read the data containing the texts
Data = pd.read_csv('C:/Users/alvar/Documents/Maestria en Ciencia de datos/Aprendizaje profundo/Proyecto final/Archivos del proyecto/files_data_DL.csv')
Data

,pdf_texts,page_number,table_number,image_number,word_number,character_number,spec_in_url,doc_type,has_text
0,Accessories (Order Separately)\nMCCAPS – Wire ...,2,3,3,1324,7707,0,other,1
1,Project: \nLocation: \nCat.No: \nType: \nLamps...,1,3,2,837,5180,0,other,1
2,Description\nSoftglo LED night lights provide ...,2,7,6,720,4217,0,spec_sheet,1
3,TruGroove surface\nTruGroove_Surface_SpecShee...,9,18,19,4366,25714,0,spec_sheet,1
4,FEATURES & SPECIFICATIONS\nINTENDED USE — Rece...,3,3,7,1101,6734,1,spec_sheet,1
...,...,...,...,...,...,...,...,...,...
496,"LFR-4RD\nLITEFRAME 4"" ROUND DOWNLIGHT\ncurrent...",11,57,23,3637,21643,1,spec_sheet,1
497,Page 1\nS4LIP TUWH PENDANT PATTERNS 01/12/2...,7,18,4,2197,13961,1,spec_sheet,1
498,A\n_ _ _ _ _ _\n_ _ _ _ _ _\n_\nSHIELDING\nCRI...,5,17,4,2118,12693,1,spec_sheet,1
499,Rook 350 Square Pendants\n LIT-2216 • 0...,9,38,5,3726,20948,1,spec_sheet,1


In [5]:
#Saves text to a list
texts = Data['pdf_texts']

Una vez teniendo los textos en una lista se meten a la función creada para poder obter los bigramas presentes en los textos, nos da los conteos de cada bigrama por texto:

In [6]:
#Gets bigrams from texts:
bow_bigram = PDFtextextract_tobow(texts, (2, 2))
bow_bigram

,00 00,00 01,00 012,00 015,00 02,00 03,00 05,00 08,00 10,00 100,...,ﬂood narrow,ﬂoor acgih,ﬂoor ceiling,ﬂoor connected,ﬂoors undoubtedly,ﬂow operat,ﬂow operating,ﬂuid collection,ﬂuorescent lamp,ﬂuorescent lighting
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Se ejecuta la función para obtener los top 10 bigramas y decidí quedarme con los 10 para mostralos, estos 10 se pueden observar en la primera parte de la impresión:

In [7]:
#Selects top 100 appearing bigrams and we keep the most relevant words
bow_bigram_reduced = BOWreduce(bow_bigram, 10)
bow_bigram_reduced

subject change     403
rights reserved    343
change notice      318
warranty year      236
lumen output       225
000 hours          223
consult factory    207
damp locations     195
120 277            193
low voltage        183
dtype: int64


,subject change,rights reserved,change notice,warranty year,lumen output,000 hours,consult factory,damp locations,120 277,low voltage
0,1,1,0,1,1,0,0,3,2,0
1,1,1,0,0,0,0,0,2,0,0
2,1,1,0,0,0,1,0,0,1,0
3,1,1,1,1,2,1,3,1,0,1
4,1,2,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
496,11,11,11,1,3,1,0,1,1,0
497,1,7,1,1,1,1,3,0,1,0
498,5,0,5,0,4,3,9,0,0,0
499,0,0,0,1,0,0,7,0,2,1
